# Imports

In [1]:
%matplotlib qt 
import pandas as pd
import backtrader_manager as bm

In [2]:
# the classes defining the backtests can be found in Bitcoin.py
from Bitcoin import (
    BenchmarkBitcoinStrategy,
    BacktestBitcoinStrategy_01, # only macd indicator
    BacktestBitcoinStrategy_02, # macd and atr indicators
    BacktestBitcoinStrategy_03, # macd and ma short/long indicators
    ) 

# Instantiate Backtest Collection 

In [3]:
# Path to pandas ohlc data
csv_datas_path = '../../datas/Bitcoin' # in this case relative path, absolute path also possible
# Name of collection
name = 'Main_collection'

# Instantiate new collection
# Or load the collection from harddrive, if it has the same name

main = bm.BacktestCollection(name=name, csv_datas_path=csv_datas_path)


Initialising Backtest Collection
✅ Loaded collection from Main_collection\pkl\collection.pkl


# Train

## Inputs

In [ ]:
# Define backtest class and paramter range for optimisation 
backtest_class = BacktestBitcoinStrategy_01
parameters_macd = {
    'MACD_slow': [20, 60], 
    'MACD_fast': [12, 24], 
    'MACD_SIGNAL': [10, 15] 
}
parameters = parameters_macd

In [ ]:
# Instatiate a new BacktestInputCollection
name_inputs = 'wfo' # wfo for walk forward optimisation

wfo_inputs = bm.BacktestInputCollection(
    name_inputs, # name that decribes type of backtests  
    max_warmup = 200, # maximum bars reserved for any indicator warmup phase prior backtest start
    window=360, # bars used for training periods
    train_perc=0.7, # defines train/test split
    backtest_class=backtest_class, # user defined subclass of bm.backtest
    strategy_parameters=parameters # parameter range of backtrader strategy 
)

In [ ]:
# Add input_collection to backtest_collection
main.add_backtest_input_collection(wfo_inputs)

In [ ]:
# Now we can still play with window size, train_perc and walkforwar_step
# earliest_start and latest end mark the maximum possible timespan that can be used for backtesting across all datafeeds

In [ ]:
wfo_inputs.window = 720

In [ ]:
wfo_inputs.train_perc = 0.7

In [ ]:
wfo_inputs.walkforward_step = int(1 * wfo_inputs.test_window)

In [ ]:
wfo_inputs.calc_train_test_periods()

In [ ]:
wfo_inputs.train_test_periods

In [ ]:
# We can also plot the periods using bars, and if we have multible datafeeds choose which close price to print)
wfo_inputs.plot_train_test_periods(use_dates=False, ref_ticker=None)

In [ ]:
# Now we create the backtest inputs of our walkforward optimisation. 
# Once we did that the train_test_periods can't be changed anymore for this input_collection
# max_chunk_size defines how many runs one single backtrader backtest max. will have

wfo_inputs.create_backtest_inputs(max_chunk_size=5)

In [ ]:
# These are our backtest_input objects that will be used in the following step
# The inputs are not connected to the input_collection, meaning changes applied to any attribute in the input collection 
# won't reflect to these inputs
len(wfo_inputs.backtest_inputs)

## Backtests

In [ ]:
# Create backtests using input collection as parameter
main.create_new_backtests(wfo_inputs)

In [ ]:
# This list stores all backtest to be run
len(main.backtests_new)

In [ ]:
main.run_backtests()

In [ ]:
# The completed backtests have been moved to backtests completed
# After every finished backtest the results are stored to harddrive and cerebro result objects are cleared from memory
len(main.backtests_completed)

In [ ]:
# The summary dataframe is the central place to examine and filter the results
# parameter_id: for each backtest_class and parameter combination a parameter_id is assigned
# idx: identifies this specific backtest/run
main.summary.head(10)

In [ ]:
# This view gives a summary overview how the groups of backtest classes and parameter set performed
main.summarize_in_groups(show_parameters=True, sort_by=('sharperatio', 'mean'))

In [ ]:
# Now we add a benchmark and some variations of the first strategy

In [ ]:
parameters_macd

In [ ]:
parameters_ma_short_long = {
    'MA_SHORT': [25, 50],
    'MA_LONG': [100, 150, 200]
}
parameters_atr = {
    'atr_period': [7, 14],
    'atr_threshold': [0.02],
}

In [ ]:
benchmark_class = BenchmarkBitcoinStrategy
parameters_benchmark = None # since its Buy and Hold

# we use same macd parameter ranges for the startegies and add additional filters
backtest_class_02 = BacktestBitcoinStrategy_02
parameters_02 = {**parameters_macd, **parameters_atr} # and add atr parameters

backtest_class_03 = BacktestBitcoinStrategy_03
parameters_03 = {**parameters_macd, **parameters_ma_short_long} # add ma short and long

In [ ]:
# Create backtes inputs for Benchmark 
wfo_inputs.backtest_class = benchmark_class
wfo_inputs.strategy_parameters = parameters_benchmark
wfo_inputs.create_backtest_inputs()

In [ ]:
# Create more backtes inputs
wfo_inputs.backtest_class = backtest_class_02
wfo_inputs.strategy_parameters = parameters_02
wfo_inputs.create_backtest_inputs()

wfo_inputs.backtest_class = backtest_class_03
wfo_inputs.strategy_parameters = parameters_03
wfo_inputs.create_backtest_inputs()

In [ ]:
# now we create all new backtests
main.create_new_backtests(wfo_inputs)

In [ ]:
# this is empty now since we used the inputs to create backtests
wfo_inputs.backtest_inputs

In [ ]:
len(main.backtests_new)

In [ ]:
# Run all new backtests
main.run_backtests()

## Selection

In [ ]:
main.summarize()
main.summary

In [ ]:
# metrics of the benchmark over all time periods
main.summarize_in_groups(backtest_class='buy_hold')

In [ ]:
# We sort the groups according to our target metric, in this case mean sharperatio over all periods
# Additionally we show  parameters 
main.summarize_in_groups(sort_by=('sharperatio', 'mean'), show_parameters=True).head(10)

In [ ]:
main.summarize_in_groups(sort_by=('cagr', 'mean')).head(5)

In [4]:
# We filter for our best ranked group
# We choose a group that performs well in terms of mean sharperatio and mean cagr
backtests_optimum = main.summarize_filter_by(input_collection_name='wfo', parameter_id='strat_03-12', show_parameters=True)

In [ ]:
# We want to inspect the equity curves for each period of our selected group
# Create quantstats reports with benchmark
main.benchmark_class = 'buy_hold'
main.load_all_results() # We first have to load all cerebro result objects from harddrive
main.get_report(summary_filtered=backtests_optimum)

# Test

In [ ]:
# our selected backtests from train period, that we now want to evaluate in the test period
backtests_optimum

In [8]:
main.backtest_input_collections[0].create_backtests_train_inputs(summary_filtered=backtests_optimum)

In [9]:
# Now lets also add our benchmark
backtests_benchmark = main.summarize_filter_by(backtest_class='buy_hold')

In [10]:
main.backtest_input_collections[0].create_backtests_train_inputs(backtests_benchmark)

In [13]:
len(main.backtest_input_collections[0].backtest_inputs)

0

In [12]:
main.create_new_backtests(main.backtest_input_collections[0])


Creating new backtests

---------------------
Initialising Backtest
---------------------
Added backtest 1 to backtests_new, mode: default

---------------------
Initialising Backtest
---------------------
Added backtest 2 to backtests_new, mode: default

---------------------
Initialising Backtest
---------------------
Added backtest 3 to backtests_new, mode: default

---------------------
Initialising Backtest
---------------------
Added backtest 4 to backtests_new, mode: default

---------------------
Initialising Backtest
---------------------
Added backtest 5 to backtests_new, mode: default

---------------------
Initialising Backtest
---------------------
Added backtest 6 to backtests_new, mode: default

---------------------
Initialising Backtest
---------------------
Added backtest 7 to backtests_new, mode: default

---------------------
Initialising Backtest
---------------------
Added backtest 8 to backtests_new, mode: default

---------------------
Initialising Backtest
---

In [14]:
main.run_backtests()

Total number of runs(combinations) to backtest: 162


Do you want to proceed? (yes/no):  y



Starting backtest 1/162 with 1 run(s)

Adding strategy datafeeds
2019-01-02 00:00:00
Added all datafeeds
Strategy added as single backtest
Initialising Strategy
2019-07-21 BACKTEST STARTING
2019-08-02 BUY ORDER PLACED
2019-08-03 BUY ORDER EXECUTED for BTCUSDT: Price: 10523.75, Value: 9000.00, Comm 4.50
2019-08-15 SELL ORDER EXECUTED for BTCUSDT: Price: 10055.16, Value: 9000.00, Comm: 4.30
2019-09-03 BUY ORDER PLACED
2019-09-04 BUY ORDER EXECUTED for BTCUSDT: Price: 10611.85, Value: 8628.62, Comm 4.31
2019-09-22 SELL ORDER EXECUTED for BTCUSDT: Price: 9985.15, Value: 8628.62, Comm: 4.06
2019-10-09 BUY ORDER PLACED
2019-10-10 BUY ORDER EXECUTED for BTCUSDT: Price: 8562.15, Value: 8166.60, Comm 4.08
2020-02-19 SELL ORDER EXECUTED for BTCUSDT: Price: 10164.78, Value: 8166.60, Comm: 4.85
2020-02-23 BACKTEST ENDING

Start Portfolio Value: 10000 
Final Portfolio Value: 10592.168881051784

Finished backtest 1 of 162
Saving BacktestCollection to Main_collection\pkl\collection.pkl
✅ Saved colle

In [15]:
main.summarize_in_groups(period_key='test')

max_drawdown  \
                                                                      max   
input_collection_name period_key backtest_class parameter_id                
wfo                   test       buy_hold       buy_hold-1      56.601326   
                                 strat_03       strat_03-12     35.869917   

                                                                         \
                                                                   mean   
input_collection_name period_key backtest_class parameter_id              
wfo                   test       buy_hold       buy_hold-1    33.532720   
                                 strat_03       strat_03-12   16.020744   

                                                                         \
                                                                    std   
input_collection_name period_key backtest_class parameter_id              
wfo                   test       buy_hold       buy_hold-1    13.575190   
                                 strat_03       strat_03-12   10.308484   

                                                                    cagr  \
                                                                    mean   
input_collection_name period_key backtest_class parameter_id               
wfo                   test       buy_hold       buy_hold-1    187.385556   
                                 strat_03       strat_03-12    98.178889   

                                                                     \
                                                             median   
input_collection_name period_key backtest_class parameter_id          
wfo                   test       buy_hold       buy_hold-1    13.09   
                                 strat_03       strat_03-12   68.10   

                                                                          \
                                                                     std   
input_collection_name period_key backtest_class parameter_id               
wfo                   test       buy_hold       buy_hold-1    420.058642   
                                 strat_03       strat_03-12   167.858568   

                                                             sharperatio  \
                                                                    mean   
input_collection_name period_key backtest_class parameter_id               
wfo                   test       buy_hold       buy_hold-1      1.000000   
                                 strat_03       strat_03-12     1.343333   

                                                                        \
                                                                   std   
input_collection_name period_key backtest_class parameter_id             
wfo                   test       buy_hold       buy_hold-1    1.648185   
                                 strat_03       strat_03-12   1.289625   

                                                             time_in_market  \
                                                                       mean   
input_collection_name period_key backtest_class parameter_id                  
wfo                   test       buy_hold       buy_hold-1       100.000000   
                                 strat_03       strat_03-12       42.037778   

                                                                         
                                                                    std  
input_collection_name period_key backtest_class parameter_id             
wfo                   test       buy_hold       buy_hold-1     0.000000  
                                 strat_03       strat_03-12   24.160939

In [18]:
main.summarize_in_groups(period_key='train', parameter_id=['strat_03-12', 'buy_hold-1'])

max_drawdown  \
                                                                      max   
input_collection_name period_key backtest_class parameter_id                
wfo                   train      buy_hold       buy_hold-1      71.680455   
                                 strat_03       strat_03-12     40.857079   

                                                                         \
                                                                   mean   
input_collection_name period_key backtest_class parameter_id              
wfo                   train      buy_hold       buy_hold-1    53.868827   
                                 strat_03       strat_03-12   27.019361   

                                                                         \
                                                                    std   
input_collection_name period_key backtest_class parameter_id              
wfo                   train      buy_hold       buy_hold-1    15.420983   
                                 strat_03       strat_03-12   11.937463   

                                                                   cagr  \
                                                                   mean   
input_collection_name period_key backtest_class parameter_id              
wfo                   train      buy_hold       buy_hold-1    72.633333   
                                 strat_03       strat_03-12   69.134444   

                                                                     \
                                                             median   
input_collection_name period_key backtest_class parameter_id          
wfo                   train      buy_hold       buy_hold-1    23.98   
                                 strat_03       strat_03-12   42.21   

                                                                          \
                                                                     std   
input_collection_name period_key backtest_class parameter_id               
wfo                   train      buy_hold       buy_hold-1    132.131354   
                                 strat_03       strat_03-12    88.732081   

                                                             sharperatio  \
                                                                    mean   
input_collection_name period_key backtest_class parameter_id               
wfo                   train      buy_hold       buy_hold-1      0.767778   
                                 strat_03       strat_03-12     1.185556   

                                                                        \
                                                                   std   
input_collection_name period_key backtest_class parameter_id             
wfo                   train      buy_hold       buy_hold-1    1.181723   
                                 strat_03       strat_03-12   0.952577   

                                                             time_in_market  \
                                                                       mean   
input_collection_name period_key backtest_class parameter_id                  
wfo                   train      buy_hold       buy_hold-1       100.000000   
                                 strat_03       strat_03-12       36.713333   

                                                                         
                                                                    std  
input_collection_name period_key backtest_class parameter_id             
wfo                   train      buy_hold       buy_hold-1     0.000000  
                                 strat_03       strat_03-12   20.207359